<a href="https://colab.research.google.com/github/NaikRyder/cte-survey/blob/main/survey_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive

In [ ]:
################# Input Layer #################

# Read Excel workbooks from gdrive

#Survey raw data
file_id="1hJx2HCXsx5z4my6LXsSgiSFjGURezfm43OXndplWPw4"
workbook_results=pd.ExcelFile(f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=xlsx")

#Key file
file_id="1R-4LQZAWZjHLXs2694YLQoetdNecDv4iwjZNNJwYbFY"
workbook_key=pd.ExcelFile(f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=xlsx")


In [ ]:
################# Transformation Layer #################

# Define Likert Scale values
likert_scale = {
  "nan" : "",
  "strongly agree" : 1,
  "agree" : 2,
  "somewhat agree" : 3,
  "neutral": 4,
  "somewhat disagree" : 5,
  "disagree" : 6,
  "strongly disagree" : 7,
  "true" : 2,
  "false" : 6,
  "somewhat true" : 3,
  "somewhat false" : 5,
  "definitely true" : 1,
  "definitely false" : 7,
  "positively true" : 1,
  "probably true" : 3,
  "probably untrue" : 5,
  "positively untrue" : 7,
  "completely agree" : 1,
  "completely disagree" : 7,
  "strongly agree, agree, somewhat agree, somewhat disagree, disagree, strongly disagree" : 4
}

# Define dataframes
df_student_key = workbook_key.parse(1)
df = {}

# Get number of sheets in the workbook and parse into a dataframe array
sheet_names = workbook_results.sheet_names
sheet_count=len(sheet_names)-1
for i in range(0,sheet_count):
  df[i]=workbook_results.parse(i)

# Data Cleaning
df_student_key['Email Address'] = df_student_key['Email Address'].str.lower()

for i in range(0,sheet_count):
    # Convert Email Address to lower case
    df[i]['Email Address'] = df[i]['Email Address'].str.lower()

    # Drop duplicate Email Addresses
    df[i] = df[i].sort_values('Timestamp').drop_duplicates(['Email Address'], keep='last')

    # Drop repeated columns in next tables
    df[i] = df[i].drop(['First Name','Last Name', 'Please give your first name and last name initial.', 'Email Address.1'], axis=1, errors='ignore')

    # Get object columns
    cols = df[i].columns[df[i].dtypes == 'object']

    # Check for text columns
    for c in cols:

      # Convert object values to lower case strings
      df[i][c] = df[i][c].astype(str).apply(str.lower)

    # Replace string with Likert Scale values
    df[i][cols] = df[i][cols].replace(likert_scale)

# Left join key with every worksheet
for i in range(0,sheet_count):
  df[i] = df_student_key.merge(df[i], how='left', on='Email Address')


In [ ]:
################# Output Layer #################

# Convert dataframe into a .csv file and save it to gdrive
drive.mount('/content/drive')
path = '/content/drive/My Drive/Survey Results/survey_data/transformed_result.xlsx'
with pd.ExcelWriter(path) as writer:
  for i in range(0,sheet_count):
    df[i].to_excel(writer, sheet_name=sheet_names[i], index=False)


Mounted at /content/drive
